In [ ]:
import cv2
import dlib
import pyttsx3
from scipy.spatial import distance

# Initializing pyttsx3 for alert audio message
engine = pyttsx3.init()

# Setting up camera to capture video from index 1
cap = cv2.VideoCapture(0)

# Face detection and mapping the face to get EYES detected
face_detector = dlib.get_frontal_face_detector()

# Load the shape predictor model for detecting landmarks
dlib_facelandmark = dlib.shape_predictor("/home/foxriver/Downloads/Downloads/practise/A.I labs/PanTech/shape_predictor_68_face_landmarks.dat")

# Function calculating the aspect ratio for the eye using Euclidean distance
def Detect_Eye(eye):
    poi_A = distance.euclidean(eye[1], eye[5])
    poi_B = distance.euclidean(eye[2], eye[4])
    poi_C = distance.euclidean(eye[0], eye[3])
    aspect_ratio_Eye = (poi_A + poi_B) / (2 * poi_C)
    return aspect_ratio_Eye

# MAIN LOOP: It will run until the program is terminated by the user
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray_scale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray_scale)

    for face in faces:
        face_landmarks = dlib_facelandmark(gray_scale, face)
        leftEye = []
        rightEye = []

        # Points allocation for the left eye
        for n in range(42, 48):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            rightEye.append((x, y))
            next_point = n + 1
            if n == 47:
                next_point = 42
            x2 = face_landmarks.part(next_point).x
            y2 = face_landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (0, 255, 0), 1)

        # Points allocation for the right eye
        for n in range(36, 42):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            leftEye.append((x, y))
            next_point = n + 1
            if n == 41:
                next_point = 36
            x2 = face_landmarks.part(next_point).x
            y2 = face_landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (255, 255, 0), 1)

        # Calculating the aspect ratio for left and right eyes
        right_Eye = Detect_Eye(rightEye)
        left_Eye = Detect_Eye(leftEye)
        Eye_Rat = (left_Eye + right_Eye) / 2

        # Round the value of the average mean of right and left eyes
        Eye_Rat = round(Eye_Rat, 2)

        # Decide whether the person's eyes are closed or not
        if Eye_Rat < 0.25:
            cv2.putText(frame, "DROWSINESS DETECTED", (50, 100), cv2.FONT_HERSHEY_PLAIN, 2, (21, 56, 210), 3)
            cv2.putText(frame, "Alert!!! WAKE UP DUDE", (50, 450), cv2.FONT_HERSHEY_PLAIN, 2, (21, 56, 212), 3)
            
            # Calling the Audio Function of Text to Audio
            engine.say("Alert WAKE UP MR")
            engine.runAndWait()

    cv2.imshow("Drowsiness Detector", frame)
    
    key = cv2.waitKey(1)
    if key == 27:  # Press 'Esc' to exit the program
        break

cap.release()
cv2.destroyAllWindows()
